# 05.5 - Evaluate learned prompts

In [ ]:
from genscai.prompts import Prompt, PromptCatalog
from genscai import paths

In [ ]:
catalog = PromptCatalog(paths.data / "prompt_catalog.db")
model_ids = catalog.retrieve_model_ids()
model_ids

In [ ]:
for model_id in model_ids:
    print(model_id)

    prompt = catalog.retrieve_last(model_id)
    print(f"version: {prompt.version}")
    print(prompt.metrics)
    print(prompt.prompt)

In [ ]:
catalog.delete_all("mistral-nemo")